In [1]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster

In [2]:
m = folium.Map(location=[14.5540,120.9752], zoom_start=10, tiles='Stamen Toner')

popup = "Manila"
manila_marker = folium.Marker([14.5995, 120.9842], popup=popup)

m.add_child(manila_marker)
m

In [5]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [ ]:
lower_left = [14.55, 120.9]
upper_right = [14.65, 121]
m = folium.Map(zoom_start = 10, location=[14.65,120.9752])
grid = get_geojson_grid(upper_right, lower_left , n=10)

for i, geo_json in enumerate(grid):

    color = plt.cm.Reds(i / len(grid))
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 1,
                                                                        'dashArray': '10, 10',
                                                                        'fillOpacity': 0.55,
                                                                    })
    popup = folium.Popup("example popup {}".format(i))
    gj.add_child(popup)

    m.add_child(gj)
m

In [3]:
df = pd.read_csv('for_map.csv')
# df['Geographic Level'] = df['Geographic Level'].fillna('City')

# from geopandas import GeoDataFrame
# gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=df['geometry'])

In [ ]:
df.info()

In [6]:
# m = folium.Map(zoom_start = 10, location=[14.5,121.25])
top_left = [14.50, 120.8]
top_right = [15, 121.21]

m = folium.Map(zoom_start = 10, location=[14.65,120.9752], tiles='Stamen Toner')


# Generate GeoJson grid
# top_right = [14, 121]
# top_left = [15, 120]
grid = get_geojson_grid(top_right, top_left , n=10)

# Calculate exposures in grid
popups = []
regional_counts = []

for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]

    mask = (
        (df.long <= upper_right[1]) & (df.long >= lower_left[1]) &
        (df.lat <= upper_right[0]) & (df.lat >= lower_left[0])
           )

    region_incidents = len(df[mask])
    regional_counts.append(region_incidents)

    total_money = df[mask].IC.sum()
    content = "total money {:,.0f}".format(total_money)
    popup = folium.Popup(content)
    popups.append(popup)

worst_region = max(regional_counts)+1

# Add GeoJson to map
for i, box in enumerate(grid):
    geo_json = json.dumps(box)

    color = plt.cm.Reds(regional_counts[i] / worst_region)
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })

    gj.add_child(popups[i])
    m.add_child(gj)


# Marker clusters
locations = list(zip(df.long, df.lat))
#icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(locations))]

# Create popups
popup_content = []
for incident in df.itertuples():
    number_of_money = "Number of vehicles: {} ".format(incident.IC)
    content = number_of_money
    popup_content.append(content)

popups = [folium.Popup(content) for content in popup_content]

cluster = MarkerCluster(locations=locations, popups=popups)
m.add_child(cluster)
# m